In [ ]:
# Welcome to your new notebook
# Type here in the cell editor to add code!


In [1]:
pip install pymongo[srv]

StatementMeta(, 4924c56e-b6d4-4b7d-be4a-aff1b0158e6d, 3, Finished, Available, Finished)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 759.5 kB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 4.3 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [7]:
# Check if PyMongo is installed
import pymongo
from bson import ObjectId
from datetime import datetime
from pymongo import MongoClient
from pyspark.sql import *
import pyspark.sql.functions as F

import json
import os
print("PyMongo version:", pymongo.version)

StatementMeta(, 4924c56e-b6d4-4b7d-be4a-aff1b0158e6d, 9, Finished, Available, Finished)

PyMongo version: 4.10.1


In [ ]:
client = MongoClient("")
db = client["production"]
#collection = db["users"]

StatementMeta(, 4924c56e-b6d4-4b7d-be4a-aff1b0158e6d, 5, Finished, Available, Finished)

In [4]:
# Directory in Lakehouse to save JSON files
output_folder = "/lakehouse/default/Files/raw_data"
os.makedirs(output_folder, exist_ok=True)  # Ensure the directory exists

# Custom JSON Encoder for MongoDB documents
class MongoJSONEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, ObjectId):
            return str(obj)  # Convert ObjectId to string
        elif isinstance(obj, datetime):
            return obj.isoformat()  # Convert datetime to ISO 8601 string
        return super().default(obj)

# Loop through each collection in the database
for collection_name in db.list_collection_names():
    collection = db[collection_name]
    
    # Fetch all documents from the collection
    documents = list(collection.find())
    
    # Save documents to a JSON file in Lakehouse
    output_file = os.path.join(output_folder, f"{collection_name}.json")
    with open(output_file, "w", encoding="utf-8") as f:
        json.dump(documents, f, indent=4, cls=MongoJSONEncoder)  # Use custom encoder
    
    print(f"Collection '{collection_name}' saved to {output_file}")

print("All collections have been saved as JSON files in the Lakehouse.")

StatementMeta(, 4924c56e-b6d4-4b7d-be4a-aff1b0158e6d, 6, Finished, Available, Finished)

Collection 'users' saved to /lakehouse/default/Files/raw_data/users.json
Collection 'busineses' saved to /lakehouse/default/Files/raw_data/busineses.json
Collection 'reviews' saved to /lakehouse/default/Files/raw_data/reviews.json
Collection 'subcategories' saved to /lakehouse/default/Files/raw_data/subcategories.json
Collection 'categories' saved to /lakehouse/default/Files/raw_data/categories.json
All collections have been saved as JSON files in the Lakehouse.


## Businesses

In [8]:
#### Processing the proudct table
json_file_path = "Files/raw_data/busineses.json"
df1 = spark.read.option("multiline", "true").json(json_file_path)

df_business = df1.withColumn('business_category', F.explode('businessCategory')).\
                withColumn('business_subcategory', F.explode('businessSubCategory')).\
                select('_id','address','business_category','business_subcategory','businessName',
                'businessPhoneNumber','city','state','claimed','contactPerson','parrotScore','createdAt',
                'businessEmail')

display(df_business)


StatementMeta(, 4924c56e-b6d4-4b7d-be4a-aff1b0158e6d, 10, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 60bbfe48-5be3-49a8-b9c7-0ff7403ad1be)

In [9]:
### to save the business file in a paruet file

df_business.write.mode('overwrite').parquet("Files/clean_data/business.parquet")

StatementMeta(, 4924c56e-b6d4-4b7d-be4a-aff1b0158e6d, 11, Finished, Available, Finished)

## Reviews

In [10]:
#### Processing the reviews table
json_file_path = "Files/raw_data/reviews.json"
df2 = spark.read.option("multiline", "true").json(json_file_path)

df2.printSchema()

StatementMeta(, 4924c56e-b6d4-4b7d-be4a-aff1b0158e6d, 12, Finished, Available, Finished)

root
 |-- __v: long (nullable = true)
 |-- _id: string (nullable = true)
 |-- business: string (nullable = true)
 |-- businessCategory: string (nullable = true)
 |-- createdAt: string (nullable = true)
 |-- dateOfExperience: string (nullable = true)
 |-- downVoters: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- downVotes: long (nullable = true)
 |-- experienceSummary: string (nullable = true)
 |-- images: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- mimeType: string (nullable = true)
 |    |    |-- url: string (nullable = true)
 |-- imagesForFlagging: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- isEdited: boolean (nullable = true)
 |-- isFlagged: boolean (nullable = true)
 |-- rating: struct (nullable = true)
 |    |-- Accessibility: long (nullable = true)
 |    |-- Appointment Flexibility: long (nullable = true)
 |    |-- Buying Experience: long (nullable = true)
 |    |-- Classes 

In [12]:
df_reviews = df2.select("_id","business","businessCategory","dateOfExperience","experienceSummary",
                        F.col("user").alias("user_id"),
                        F.col("rating.Accessibility").alias("Accessibility"),
                        F.col("rating.Appointment Flexibility").alias("Appointment Flexibility"),
                        F.col("rating.Buying Experience").alias("Buying Experience"),
                        F.col("rating.Classes & Programs").alias("Classes & Programs"),
                        F.col("rating.Cleanliness & Ambience").alias("Cleanliness & Ambience"),
                        F.col("rating.Customer Service").alias("Customer Service"),
                        F.col("rating.Equipment Quality").alias("Equipment Quality"),
                        F.col("rating.Food & Dining").alias("Food & Dining"),
                        F.col("rating.Food Quality").alias("Food Quality"),
                        F.col("rating.Learning Experience").alias("Learning Experience"),
                        F.col("rating.Networking & Interaction").alias("Networking & Interaction"),
                        F.col("rating.Organization").alias("Organization"),
                        F.col("rating.Product Quality").alias("Product Quality"),
                        F.col("rating.Professionalism").alias("Professionalism"),
                        F.col("rating.Quality of Care").alias("Quality of Care"),
                        F.col("rating.Quality of Curriculum").alias("Quality of Curriculum"),
                        F.col("rating.Quality of Instructors").alias("Quality of Instructors"),
                        F.col("rating.Quality of Service").alias("Quality of Service"),
                        F.col("rating.Room Quality").alias("Room Quality"),
                        F.col("rating.Safety & Security").alias("Safety & Security"),
                        F.col("rating.Security").alias("Security"),
                        F.col("rating.Service Quality").alias("Service Quality"),
                        F.col("rating.Session Quality").alias("Session Quality"),
                        F.col("rating.Timeliness").alias("Timeliness"),
                        F.col("rating.Wait Time").alias("Wait Time"))



display(df_reviews)

StatementMeta(, 4924c56e-b6d4-4b7d-be4a-aff1b0158e6d, 14, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 91f44049-f56e-4dc1-b34c-c0f3b056273e)

In [13]:
### to save the reviews file in a paruet file

df_reviews.write.mode('overwrite').parquet("Files/clean_data/reviews.parquet")

StatementMeta(, 4924c56e-b6d4-4b7d-be4a-aff1b0158e6d, 15, Finished, Available, Finished)

In [23]:
#spark.read.parquet("Files/clean_data/reviews.parquet")

StatementMeta(, 8c9b19a9-ef64-4dcc-8fae-5a553eea8f96, 25, Finished, Available, Finished)

# Categories

In [5]:
#### Processing the categories table
json_file_path = "Files/raw_data/categories.json"
df3 = spark.read.option("multiline", "true").json(json_file_path)

display(df3)

StatementMeta(, a306c6dc-82e4-4968-99ca-8e46d9efd520, 7, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 35a7f2f4-3cf6-4d07-8281-683a2b110694)

In [12]:
df_categories = df3.withColumn('metrics', F.explode('metrics')).\
                 select('_id','metrics','name','slug','status')
display(df_categories)

StatementMeta(, a306c6dc-82e4-4968-99ca-8e46d9efd520, 14, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, a6d80e2a-55ed-4402-b667-cbe2f5f24141)

In [14]:
### to save the categories file in a paruet file

df_categories.write.mode('overwrite').parquet("Files/clean_data/categories.parquet")

StatementMeta(, a306c6dc-82e4-4968-99ca-8e46d9efd520, 16, Finished, Available, Finished)

# Sub-categories

In [5]:
#### Processing the categories table
json_file_path = "Files/raw_data/subcategories.json"
df4 = spark.read.option("multiline", "true").json(json_file_path)

display(df4)

StatementMeta(, acab9323-e318-4b23-a069-24809c12582b, 7, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 3299ac6c-be32-40b2-933a-a765c6b52e04)

In [6]:
df_subcategories = df4.withColumn('categories_id', F.explode('categoryIds')).\
                 select('_id','categories_id','name','slug','status','description')
display(df_subcategories)

StatementMeta(, acab9323-e318-4b23-a069-24809c12582b, 8, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, b79f3206-8f61-48c8-a7b1-ce4e420ccdf1)

In [7]:
### to save the subcategories file in a paruet file

df_subcategories.write.mode('overwrite').parquet("Files/clean_data/subcategories.parquet")

StatementMeta(, acab9323-e318-4b23-a069-24809c12582b, 9, Finished, Available, Finished)

# Users

In [8]:
#### Processing the categories table
json_file_path = "Files/raw_data/users.json"
df5 = spark.read.option("multiline", "true").json(json_file_path)

display(df5)

StatementMeta(, acab9323-e318-4b23-a069-24809c12582b, 10, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 0edceb73-248f-4ac9-91fe-4d4a23aac029)

In [9]:
df_users = df5.select('_id','firstName','lastName','gender','state','username','city','email','verificationStatus')
display(df_users)

StatementMeta(, acab9323-e318-4b23-a069-24809c12582b, 11, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 626d52ad-69dc-4424-94d0-956256ce62a2)

In [10]:
### to save the users file in a paruet file

df_users.write.mode('overwrite').parquet("Files/clean_data/users.parquet")

StatementMeta(, acab9323-e318-4b23-a069-24809c12582b, 12, Finished, Available, Finished)